# UNSW-NB15: Feature Engineering

In [1]:
import numpy as np  # for array
import pandas as pd  # for csv files and dataframe
import matplotlib.pyplot as plt  # for plotting
import seaborn as sns  # plotting
from scipy import stats

import pickle  # To load data int disk
from prettytable import PrettyTable  # To print in tabular format

import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer
from sklearn.metrics import auc, f1_score, roc_curve
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_validate, cross_val_predict

%matplotlib inline

In [2]:
file_path = "./final_ipynb/"

In [3]:
saved_dict = pickle.load(open(file_path+'saved_dict.pkl', 'rb'))

In [4]:
# Loading data from disk
train = pd.read_csv(file_path+'train_alldata_EDA.csv')
test = pd.read_csv(file_path+'test_alldata_EDA.csv')

In [5]:
# Utility function
def multi_corr(col1, col2="label", df=train):
    '''
    This function returns correlation between 2 given features.
    Also gives corr of the given features with "label" afetr applying log1p to it.
    '''
    corr = df[[col1, col2]].corr().iloc[0,1]
    log_corr = df[col1].apply(np.log1p).corr(df[col2])

    print("Correlation : {}\nlog_Correlation: {}".format(corr, log_corr))

In [6]:
def corr(col1, col2="label", df=train):
    """
    This function returns correlation between 2 given features
    """
    return df[[col1, col2]].corr().iloc[0,1]

## Removing highly correlated features

In [7]:
# Selecting all the features with high correlation values with other features
# Refer: https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
corr_matrix = train.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find index of feature columns with correlation greater than 0.9
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [8]:
# We don't want to use these features for plotting because these are having high corr
# And most likely have same kind of plots with already plotted feature
print(to_drop)

['dloss', 'dpkts', 'dwin', 'ltime', 'ct_src_dport_ltm']


In [9]:
saved_dict['corr_col'] = to_drop

In [10]:
# removing the features from train and test data
train.drop(columns=to_drop, inplace=True)

In [11]:
train.shape, test.shape

((2032037, 44), (508010, 49))

## Adding New Features

Refer: https://www.elastic.co/guide/en/ecs/master/ecs-network.html
* Network bytes: Total bytes trasferred by the network. It is sum of 'sbytes' (Source to destination bytes) and 'dbytes' (Destination to source bytes).

In [12]:
# creating new features
train['network_bytes'] = train['sbytes'] + train['dbytes']

In [13]:
train.shape, test.shape

((2032037, 45), (508010, 49))

In [14]:
# Dropping columns which are not useful for the classification
# attack_cat is for multiclass classification
# all the other columns are address related and not present in sample train data
train.drop(['srcip', 'sport', 'dstip', 'dsport', 'attack_cat'], axis=1, inplace=True)

In [15]:
# To use during test data transformation
saved_dict['to_drop'] = ['srcip', 'sport', 'dstip', 'dsport', 'attack_cat']

In [16]:
train.shape, test.shape

((2032037, 40), (508010, 49))

## Applying log1p on Numerical columns

During EDA we found that few numerical columns shows better visualization for pdf curves if we apply log1p to the columns.

So I thought to try log1p on all the columns and check the correlation value of the original column and log1p column with target column i.e. "label"

In [17]:
# Getting number of unique values of all the columns
# If the unique values are high that means it has continuous set of values
col_unique_values = train.nunique()

In [18]:
# If the unique values are getter than some threshould than we will check its corr
col = col_unique_values[col_unique_values>200].index

In [19]:
# Checking corr value of original col and log1p applied col
# Taking those columns whose unique values are getter than some threshould
for column in col:
    print("{:-^30}".format(column))
    multi_corr(column)

-------------dur--------------
Correlation : -0.0002745058290609313
log_Correlation: -0.03693554176498668
------------sbytes------------
Correlation : 0.010095279709733698
log_Correlation: -0.3405661153969544
------------dbytes------------
Correlation : -0.07184729733702347
log_Correlation: -0.5205853978490864
------------sloss-------------
Correlation : -0.043374126008650975
log_Correlation: -0.29101957669235745
------------sload-------------
Correlation : 0.21477885640137434
log_Correlation: 0.3510714305228587
------------dload-------------
Correlation : -0.20882598322673818
log_Correlation: -0.6139366298720378
------------spkts-------------
Correlation : -0.1170186810172029
log_Correlation: -0.30592842917677887
------------stcpb-------------
Correlation : -0.2226408378141495
log_Correlation: -0.30279805875915294
------------dtcpb-------------
Correlation : -0.22255599540386262
log_Correlation: -0.3026023459994278
-----------smeansz------------
Correlation : -0.059085556417158416
log

In [20]:
# Will apply log1p on this columns and remove original columns
log1p_col = ['dur', 'sbytes', 'dbytes', 'sload', 'dload', 'spkts', 'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'sjit', 'djit', 'network_bytes']

In [21]:
saved_dict['log1p_col'] = log1p_col

In [22]:
# mode values of every features, will use to fill Null values of test
mode_dict = train.mode().iloc[0].to_dict()

In [23]:
def log1p_transform(col, df=train):
    '''
    Apply log1p on given column.
    Remove the original cola and keep log1p applied col
    '''
    new_col = col+'_log1p'
    df[new_col] = df[col].apply(np.log1p)
    df.drop(col, axis=1, inplace=True)

In [24]:
# Transforming columns with log1p
for col in log1p_col:
    log1p_transform(col, df=train)

In [25]:
train.shape

(2032037, 40)

In [26]:
train.columns

Index(['proto', 'state', 'sttl', 'dttl', 'sloss', 'service', 'swin',
       'trans_depth', 'res_bdy_len', 'stime', 'sintpkt', 'dintpkt', 'tcprtt',
       'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
       'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
       'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm', 'ct_dst_sport_ltm',
       'ct_dst_src_ltm', 'label', 'dur_log1p', 'sbytes_log1p', 'dbytes_log1p',
       'sload_log1p', 'dload_log1p', 'spkts_log1p', 'stcpb_log1p',
       'dtcpb_log1p', 'smeansz_log1p', 'dmeansz_log1p', 'sjit_log1p',
       'djit_log1p', 'network_bytes_log1p'],
      dtype='object')

In [27]:
train.shape, test.shape

((2032037, 40), (508010, 49))

In [28]:
# creating x and y set from the dataset
x_train, y_train = train.drop(columns=['label']), train['label']
x_test, y_test = test.drop(columns=['label']), test['label']

In [29]:
print(x_train.shape, y_train.shape)
print()
print(x_test.shape, y_test.shape)

(2032037, 39) (2032037,)

(508010, 48) (508010,)


In [30]:
# getting categorical and numerical columns in 2 diff lists
cat_col = ['proto', 'service', 'state']
num_col = list(set(x_train.columns) - set(cat_col))

In [31]:
# To use later, during test data cleaning
saved_dict['cat_col'] = cat_col
saved_dict['num_col'] = num_col

In [32]:
x_train.head()

,proto,state,sttl,dttl,sloss,service,swin,trans_depth,res_bdy_len,stime,...,sload_log1p,dload_log1p,spkts_log1p,stcpb_log1p,dtcpb_log1p,smeansz_log1p,dmeansz_log1p,sjit_log1p,djit_log1p,network_bytes_log1p
0,udp,CON,31,29,0,dns,0,0,0,1421927414,...,13.123313,13.340377,1.098612,0.0,0.0,4.204693,4.418841,0.000000,0.000000,5.693732
1,udp,CON,31,29,0,None,0,0,0,1421927414,...,11.381416,10.829356,1.609438,0.0,0.0,4.890349,4.343805,2.387938,2.458112,6.725034
2,udp,CON,31,29,0,dns,0,0,0,1421927414,...,13.165223,13.363399,1.098612,0.0,0.0,4.304065,4.499810,0.000000,0.000000,5.783825
3,udp,CON,31,29,0,dns,0,0,0,1421927414,...,12.987060,13.204124,1.098612,0.0,0.0,4.204693,4.418841,0.000000,0.000000,5.693732
4,udp,CON,31,29,0,dns,0,0,0,1421927414,...,13.121510,13.319686,1.098612,0.0,0.0,4.304065,4.499810,0.000000,0.000000,5.783825


## Standardizing
    
As we have seen that the range of few features in this dataset is very large. So we will keep everything within certain range by applying standardscaler. After this all the features will have mean 0 and std 1

In [33]:
# Standardizing the data
scaler = StandardScaler()
scaler = scaler.fit(x_train[num_col])

In [34]:
x_train[num_col] = scaler.transform(x_train[num_col])

In [35]:
x_train.head()

,proto,state,sttl,dttl,sloss,service,swin,trans_depth,res_bdy_len,stime,...,sload_log1p,dload_log1p,spkts_log1p,stcpb_log1p,dtcpb_log1p,smeansz_log1p,dmeansz_log1p,sjit_log1p,djit_log1p,network_bytes_log1p
0,udp,CON,-0.387689,-0.04671,-0.247087,dns,-1.25412,-0.244819,-0.091917,-0.951488,...,-0.101337,0.407580,-1.128518,-1.248081,-1.247712,-0.526652,-0.018948,-1.029888,-0.933149,-1.075803
1,udp,CON,-0.387689,-0.04671,-0.247087,None,-1.25412,-0.244819,-0.091917,-0.951488,...,-0.679554,-0.060217,-0.752408,-1.248081,-1.247712,0.563425,-0.053225,-0.289198,-0.063615,-0.630911
2,udp,CON,-0.387689,-0.04671,-0.247087,dns,-1.25412,-0.244819,-0.091917,-0.951488,...,-0.087425,0.411869,-1.128518,-1.248081,-1.247712,-0.368667,0.018038,-1.029888,-0.933149,-1.036937
3,udp,CON,-0.387689,-0.04671,-0.247087,dns,-1.25412,-0.244819,-0.091917,-0.951488,...,-0.146566,0.382196,-1.128518,-1.248081,-1.247712,-0.526652,-0.018948,-1.029888,-0.933149,-1.075803
4,udp,CON,-0.387689,-0.04671,-0.247087,dns,-1.25412,-0.244819,-0.091917,-0.951488,...,-0.101936,0.403725,-1.128518,-1.248081,-1.247712,-0.368667,0.018038,-1.029888,-0.933149,-1.036937


## Onehot Encoding

In our dataset we have few categorical columns with text data.
But ML models can't process text data it can process numbers.

So we have to convert categorical columns to numerical columns in some way.
We will use onehotencoder where we will assign 1 if the value is present for the row and rest of the columns will be 0.

In [36]:
# Onehot Encoding
service_ = OneHotEncoder()
proto_ = OneHotEncoder()
state_ = OneHotEncoder()
ohe_service = service_.fit(x_train.service.values.reshape(-1,1))
ohe_proto = proto_.fit(x_train.proto.values.reshape(-1,1))
ohe_state = state_.fit(x_train.state.values.reshape(-1,1))

In [37]:
# We are onehot encoding the given column
# Remove the original categorical column
for col, ohe in zip(['proto', 'service', 'state'], [ohe_proto, ohe_service, ohe_state]):
    x = ohe.transform(x_train[col].values.reshape(-1,1))
    tmp_df = pd.DataFrame(x.todense(), columns=[col+'_'+i for i in ohe.categories_[0]])
    x_train = pd.concat([x_train.drop(col, axis=1), tmp_df], axis=1)

In [38]:
x_train.head()

,sttl,dttl,sloss,swin,trans_depth,res_bdy_len,stime,sintpkt,dintpkt,tcprtt,...,state_INT,state_MAS,state_PAR,state_REQ,state_RST,state_TST,state_TXD,state_URH,state_URN,state_no
0,-0.387689,-0.04671,-0.247087,-1.25412,-0.244819,-0.091917,-0.951488,-0.071862,-0.056309,-0.137899,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.387689,-0.04671,-0.247087,-1.25412,-0.244819,-0.091917,-0.951488,-0.069458,-0.051276,-0.137899,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.387689,-0.04671,-0.247087,-1.25412,-0.244819,-0.091917,-0.951488,-0.071862,-0.056309,-0.137899,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.387689,-0.04671,-0.247087,-1.25412,-0.244819,-0.091917,-0.951488,-0.071853,-0.056308,-0.137899,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.387689,-0.04671,-0.247087,-1.25412,-0.244819,-0.091917,-0.951488,-0.071866,-0.056315,-0.137899,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Saving all the important parameters and objects to disk so that we can apply same process on test data

In [39]:
file_path = './final_ipynb/'

In [40]:
pickle.dump(scaler, open(file_path+'scaler.pkl', 'wb'))  # Standard scaler
pickle.dump(saved_dict, open(file_path+'saved_dict.pkl', 'wb'))  # Dictionary with important parameters
pickle.dump(mode_dict, open(file_path+'mode_dict.pkl', 'wb'))  #  Dictionary with most frequent values of columns

In [41]:
# Onehot encoder for categorical columns
pickle.dump(ohe_proto, open(file_path+'ohe_proto.pkl', 'wb'))
pickle.dump(ohe_service, open(file_path+'ohe_service.pkl', 'wb'))
pickle.dump(ohe_state, open(file_path+'ohe_state.pkl', 'wb'))

## Pipeline functions

We have to prepare a pipeline, where we can send raw data and get the output.

We will use test data to implement the pipeline. Here we will use all the parameters we have saved using train data.

Also standardize and onehot encode test data using train data objects for standardscaler and onehotencoder.

In [42]:
def clean_data(data):
    '''
    Cleans given raw data. Performs various cleaning, removes Null and wrong values.
    Check for columns datatype and fix them.
    '''
    numerical_col = data.select_dtypes(include=np.number).columns  # All the numerical columns list
    categorical_col = data.select_dtypes(exclude=np.number).columns  # All the categorical columns list
    
    # Cleaning the data
    for col in data.columns:
        val = mode_dict[col]  # Mode value of the column in train data
        data[col] = data[col].fillna(value=val)
        data[col] = data[col].replace(' ', value=val)
        data[col] = data[col].apply(lambda x:"None" if x=="-" else x)

        # Fixing binary columns
        if col in saved_dict['binary_col']:
            data[col] = np.where(data[col]>1, val, data[col])

    # Fixing datatype of columns
    bad_dtypes = list(set(categorical_col) - set(saved_dict['cat_col']))
    for bad_col in bad_dtypes:
        data[col] = data[col].astype(float)
    
    return data

In [43]:
def apply_log1p(data):
    '''
    Performs FE on the data. Apply log1p on the specified columns create new column and remove those original columns.
    '''
    for col in saved_dict['log1p_col']:
        new_col = col + '_log1p'  # New col name
        data[new_col] = data[col].apply(np.log1p)  # Creating new column on transformed data
        data.drop(col, axis=1, inplace=True)  # Removing old columns
    return data

In [44]:
def standardize(data):
    '''
    Stanardize the given data. Performs mean centering and varience scaling.
    Using stanardscaler object trained on train data.
    '''
    data[saved_dict['num_col']] = scaler.transform(data[saved_dict['num_col']])
    return data

In [45]:
def ohencoding(data):
    '''
    Onehot encoding the categoricla columns.
    Add the ohe columns with the data and removes categorical columns.
    Using Onehotencoder objects trained on train data.
    '''

    # Onehot encoding cat col using onehotencoder objects
    X = ohe_service.transform(data['service'].values.reshape(-1, 1))
    Xm = ohe_proto.transform(data['proto'].values.reshape(-1, 1))
    Xmm = ohe_state.transform(data['state'].values.reshape(-1, 1))
    
    # Adding encoding data to original data
    data = pd.concat([data,
                      pd.DataFrame(Xm.toarray(), columns=['proto_'+i for i in ohe_proto.categories_[0]]),
                      pd.DataFrame(X.toarray(), columns=['service_'+i for i in ohe_service.categories_[0]]),
                      pd.DataFrame(Xmm.toarray(), columns=['state_'+i for i in ohe_state.categories_[0]])],
                      axis=1)
    
    # Removing cat columns
    data.drop(['proto', 'service', 'state'], axis=1, inplace=True)

    return data

Loading all the objects from disk, that we have trained on train data.

In [46]:
# Parametrs
saved_dict = pickle.load(open(file_path+'saved_dict.pkl', 'rb'))
# Mode value of all the columns
mode_dict = pickle.load(open(file_path+'mode_dict.pkl', 'rb'))
# Stanardscaler object
scaler = pickle.load(open(file_path+'scaler.pkl', 'rb'))

In [47]:
# One hot encoder objects
ohe_proto = pickle.load(open(file_path+'ohe_proto.pkl', 'rb'))
ohe_service = pickle.load(open(file_path+'ohe_service.pkl', 'rb'))
ohe_state = pickle.load(open(file_path+'ohe_state.pkl', 'rb'))

In [48]:
x_test.shape

(508010, 48)

In [49]:
# Resetting index of test data
x_test.reset_index(drop=True, inplace=True)

In [50]:
x_test.shape

(508010, 48)

In [51]:
x_test.columns

Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'sload', 'dload',
       'spkts', 'dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'sjit', 'djit', 'stime',
       'ltime', 'sintpkt', 'dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
       'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat'],
      dtype='object')

In [52]:
# Adding column names
x_test.columns = saved_dict['columns']

In [53]:
# Creating new Feature
x_test['network_bytes'] = x_test['dbytes'] + x_test['sbytes']

In [54]:
# Droping all the unwanted columns
dropable_col = saved_dict['to_drop'] + saved_dict['corr_col']
x_test.drop(columns=dropable_col, inplace=True)

In [55]:
x_test.shape

(508010, 39)

In [56]:
# Cleaning data using clean_data()
x_test = clean_data(x_test)

In [57]:
x_test.shape

(508010, 39)

In [58]:
# FE: applying log1p using apply_log1p()
x_test = apply_log1p(x_test)

In [59]:
x_test.shape

(508010, 39)

In [60]:
x_test.columns

Index(['proto', 'state', 'sttl', 'dttl', 'sloss', 'service', 'swin',
       'trans_depth', 'res_bdy_len', 'stime', 'sintpkt', 'dintpkt', 'tcprtt',
       'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl',
       'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src',
       'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm', 'ct_dst_sport_ltm',
       'ct_dst_src_ltm', 'dur_log1p', 'sbytes_log1p', 'dbytes_log1p',
       'sload_log1p', 'dload_log1p', 'spkts_log1p', 'stcpb_log1p',
       'dtcpb_log1p', 'smeansz_log1p', 'dmeansz_log1p', 'sjit_log1p',
       'djit_log1p', 'network_bytes_log1p'],
      dtype='object')

In [61]:
# Standardscaling using stanardize()
x_test = standardize(x_test)

In [62]:
x_test.head()

,proto,state,sttl,dttl,sloss,service,swin,trans_depth,res_bdy_len,stime,...,sload_log1p,dload_log1p,spkts_log1p,stcpb_log1p,dtcpb_log1p,smeansz_log1p,dmeansz_log1p,sjit_log1p,djit_log1p,network_bytes_log1p
0,udp,INT,0.025056,-0.766532,-0.247087,dns,-1.254120,-0.244819,-0.091917,1.083818,...,2.209440,-2.077696,-1.128518,-1.248081,-1.247712,0.563425,-2.037488,-1.029888,-0.933149,-1.124982
1,udp,INT,0.025056,-0.766532,-0.247087,dns,-1.254120,-0.244819,-0.091917,1.083818,...,2.209440,-2.077696,-1.128518,-1.248081,-1.247712,0.563425,-2.037488,-1.029888,-0.933149,-1.124982
2,udp,INT,0.025056,-0.766532,-0.247087,dns,-1.254120,-0.244819,-0.091917,1.083818,...,2.209440,-2.077696,-1.128518,-1.248081,-1.247712,0.563425,-2.037488,-1.029888,-0.933149,-1.124982
3,tcp,FIN,-0.387689,-0.046710,-0.109351,http,0.797381,2.655285,-0.091917,1.083817,...,-1.352704,0.017172,-0.048886,0.862893,0.751843,0.563425,0.858001,1.812003,2.231315,0.510753
4,tcp,FIN,-0.387689,-0.046710,0.074297,None,0.797381,-0.244819,-0.091917,1.083818,...,0.025248,0.972515,1.383845,0.882854,0.805791,-0.755985,1.113866,0.085242,0.328498,1.418027


In [63]:
# Onehot encoding categorical columns using ohencoding()
x_test = ohencoding(x_test)

In [64]:
x_test.shape

(508010, 200)

In [65]:
# Final test data
x_test.head()

,sttl,dttl,sloss,swin,trans_depth,res_bdy_len,stime,sintpkt,dintpkt,tcprtt,...,state_INT,state_MAS,state_PAR,state_REQ,state_RST,state_TST,state_TXD,state_URH,state_URN,state_no
0,0.025056,-0.766532,-0.247087,-1.254120,-0.244819,-0.091917,1.083818,-0.071867,-0.056317,-0.137899,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.025056,-0.766532,-0.247087,-1.254120,-0.244819,-0.091917,1.083818,-0.071867,-0.056317,-0.137899,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.025056,-0.766532,-0.247087,-1.254120,-0.244819,-0.091917,1.083818,-0.071867,-0.056317,-0.137899,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.387689,-0.046710,-0.109351,0.797381,2.655285,-0.091917,1.083817,-0.040457,-0.016952,-0.120097,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.387689,-0.046710,0.074297,0.797381,-0.244819,-0.091917,1.083818,-0.071650,-0.055912,-0.122230,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
# Matching test data columns with train data columns
all(x_train.columns == x_test.columns)

True

In [68]:
# Cleaned an processed train data
x_train.to_csv(file_path+'x_train.csv', index=False)
y_train.to_csv(file_path+'y_train.csv', index=False)

# Cleaned an processed test data
x_test.to_csv(file_path+'x_test.csv', index=False)
y_test.to_csv(file_path+'y_test.csv', index=False)